In [1]:
import ee
import numpy as np
import pandas as pd

In [2]:
ee.Authenticate()


True

In [3]:
ee.Initialize(project='ee-apr160')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [4]:
df_data = pd.read_csv('data/2023.csv',encoding = "ISO-8859-1")
df_data = df_data[['Latitud','Longitud']].drop_duplicates().reset_index(drop=True)
#df_data = df_data.iloc[215:]

df_data.to_csv('points.csv')

In [8]:
def add_date(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def save_coordinate(Sentinel_data,row, feature):
    features = []
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
    poi_properties = dict(row)
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    features.append(poi_feature)
    ee_fc = ee.FeatureCollection(features) 


    def rasterExtraction(image):
        feature = image.sampleRegions(
            collection = ee_fc, # feature collection here
            scale = 10 # Cell size of raster
        )
        return feature
    
    results = Sentinel_data.filterBounds(ee_fc).select(feature).map(add_date).map(rasterExtraction).flatten()

    # You can modify this for better optimization
    column_df = list(df_data.columns)
    column_df.extend([feature, 'date'])
    
    nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()
    df_final = pd.DataFrame(data,columns=column_df)
    df_final['date'] = pd.to_datetime(df_final['date'],format='%Y%m%d')  
    df_final = df_final.set_index('date').asfreq('D', method='ffill').reset_index()
    df_final.to_csv('gee-coordinates-2023/'+feature+'/' + str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
   

def get_data(source,from_date,to_date,feature):
    Sentinel_data = ee.ImageCollection(source) \
    .filterDate(from_date,to_date) \
    .map(add_date)

    for index, row in df_data.iterrows():
        save_coordinate(Sentinel_data,row,feature)
        print('.',end='')

In [10]:
get_data('MODIS/061/MYD13Q1',"2022-09-04","2023-01-16",'NDVI')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [11]:
get_data('MODIS/061/MOD11A1',"2022-09-04","2023-01-16",'LST_Day_1km')


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................